In [1]:
%run ../yp_utils.py

# Initial setup

In [2]:
paper_pmid = 30377274
paper_name = 'perez_samper_verstrepen_2018' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [5]:
original_data = pd.read_excel('raw_data/RawData_BarSeq_Gemma.xlsx', sheet_name='Calculate_extendedGRs', skiprows=23)

In [6]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 4997 x 125


In [7]:
original_data.head()

,Unnamed: 0,Unnamed: 1,NameGen,NameTriv,Class_Stab_Glu_Up_A,Class_Stab_Glu_Up_B,Class_Stab_Gal_Up_A,Class_Stab_Gal_Up_B,Class_Grad_Gal_Up_A,Class_Grad_Gal_Up_B,...,NameGen.5,NameTriv.5,ExtRatio_Stab_Gal_Up_A.1,ExtRatio_Stab_Gal_Up_B.1,ExtRatio_Grad_Gal_Up_A.1,ExtRatio_Grad_Gal_Up_B.1,ExtRatio_Stab_Gal_Dn_A.1,ExtRatio_Stab_Gal_Dn_B.1,ExtRatio_Grad_Gal_Dn_A.1,ExtRatio_Grad_Gal_Dn_B.1
0,NaN,NaN,YAL002W,VPS8,A,A,A,A,A,A,...,YAL002W,VPS8,0.557287,0.612309,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,YAL004W,YAL004W,A,A,A,A,A,A,...,YAL004W,YAL004W,0.686534,0.706888,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,YAL005C,SSA1,A,A,A,A,A,A,...,YAL005C,SSA1,0.687902,0.699443,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,YAL007C,ERP2,A,A,A,A,A,A,...,YAL007C,ERP2,0.687088,0.693878,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,YAL008W,FUN14,A,A,A,A,A,A,...,YAL008W,FUN14,0.664669,0.688202,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
original_data['orf'] = original_data['NameGen'].astype(str)

In [18]:
original_data['orf'] = original_data['orf'].apply(lambda x: x.replace('_','-'))

In [19]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [20]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [21]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [Unnamed: 0, Unnamed: 1, NameGen, NameTriv, Class_Stab_Glu_Up_A, Class_Stab_Glu_Up_B, Class_Stab_Gal_Up_A, Class_Stab_Gal_Up_B, Class_Grad_Gal_Up_A, Class_Grad_Gal_Up_B, Class_Stab_Glu_Dn_A, Class_Stab_Glu_Dn_B, Class_Stab_Gal_Dn_A, Class_Stab_Gal_Dn_B, Class_Grad_Gal_Dn_A, Class_Grad_Gal_Dn_B, GR_Stab_Glu_Up_A, GR_Stab_Glu_Up_B, GR_Stab_Gal_Up_A, GR_Stab_Gal_Up_B, GR_Grad_Gal_Up_A, GR_Grad_Gal_Up_B, GR_Stab_Glu_Dn_A, GR_Stab_Glu_Dn_B, GR_Stab_Gal_Dn_A, GR_Stab_Gal_Dn_B, GR_Grad_Gal_Dn_A, GR_Grad_Gal_Dn_B, Unnamed: 28, NameGen.1, NameTriv.1, InitFreq_Stab_Up, Unnamed: 32, Unnamed: 33, Unnamed: 34, Unnamed: 35, Unnamed: 36, InitFreq_Grad_Up, Unnamed: 38, Unnamed: 39, Unnamed: 40, Unnamed: 41, Unnamed: 42, InitFreq_Stab_Dn, Unnamed: 44, Unnamed: 45, Unnamed: 46, Unnamed: 47, Unnamed: 48, InitFreq_Grad_Dn, Unnamed: 50, Unnamed: 51, Unnamed: 52, Unnamed: 53, Unnamed: 54, Unnamed: 55, NameGen.2, NameTriv.2, FinalFreq_Stab_Glu_Up_A, FinalFreq_Stab_Glu_Up_B, FinalFreq

In [22]:
data_cols = [c for c in original_data.columns if c.startswith('GR')]

In [23]:
data_cols

['GR_Stab_Glu_Up_A',
 'GR_Stab_Glu_Up_B',
 'GR_Stab_Gal_Up_A',
 'GR_Stab_Gal_Up_B',
 'GR_Grad_Gal_Up_A',
 'GR_Grad_Gal_Up_B',
 'GR_Stab_Glu_Dn_A',
 'GR_Stab_Glu_Dn_B',
 'GR_Stab_Gal_Dn_A',
 'GR_Stab_Gal_Dn_B',
 'GR_Grad_Gal_Dn_A',
 'GR_Grad_Gal_Dn_B']

In [24]:
original_data.set_index('orf', inplace=True)

In [25]:
original_data = original_data[data_cols].copy()

In [29]:
# Average A and B, Up and Down
cs = ['_'.join(c.split('_')[:3]) for c in data_cols]

In [31]:
original_data.columns = cs
original_data = original_data.groupby(original_data.columns, axis=1).mean()

In [32]:
original_data.shape

(4997, 3)

In [33]:
original_data = original_data.groupby(original_data.index).mean()

In [34]:
original_data.shape

(4961, 3)

In [35]:
original_data.head()

,GR_Grad_Gal,GR_Stab_Gal,GR_Stab_Glu
orf,,,
YAL002W,0.129686,0.136258,0.233565
YAL004W,0.128749,0.169507,0.243335
YAL005C,0.130646,0.169959,0.245045
YAL007C,0.139847,0.186705,0.270419
YAL008W,0.134678,0.169771,0.251057


In [48]:
# Normalize gradual shift to glucose 5%
original_data['GR_Grad_Gal'] = original_data['GR_Grad_Gal'] / original_data['GR_Stab_Glu']

# Prepare the final dataset

In [49]:
data = original_data.copy()

In [50]:
dataset_ids = [21907, 21961, 21960]
datasets = datasets.reindex(index=dataset_ids)

In [51]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [52]:
data.head()

dataset_id,21907,21961,21960
data_type,value,value,value
orf,,,
YAL002W,0.555246,0.136258,0.233565
YAL004W,0.529100,0.169507,0.243335
YAL005C,0.533151,0.169959,0.245045
YAL007C,0.517149,0.186705,0.270419
YAL008W,0.536442,0.169771,0.251057


## Subset to the genes currently in SGD

In [53]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 23


In [54]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,21907,21961,21960
,data_type,value,value,value
gene_id,orf,,,
2,YAL002W,0.555246,0.136258,0.233565
1863,YAL004W,0.529100,0.169507,0.243335
4,YAL005C,0.533151,0.169959,0.245045
5,YAL007C,0.517149,0.186705,0.270419
6,YAL008W,0.536442,0.169771,0.251057


# Normalize

In [55]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [56]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [57]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

,dataset_id,21907,21961,21960,21907,21961,21960
,data_type,value,value,value,valuez,valuez,valuez
gene_id,orf,,,,,,
2,YAL002W,0.555246,0.136258,0.233565,0.629761,-3.277333,-1.009457
1863,YAL004W,0.529100,0.169507,0.243335,-0.332166,-0.165697,-0.457754
4,YAL005C,0.533151,0.169959,0.245045,-0.183133,-0.123391,-0.361201
5,YAL007C,0.517149,0.186705,0.270419,-0.771839,1.443754,1.071597
6,YAL008W,0.536442,0.169771,0.251057,-0.062047,-0.141035,-0.021709


# Print out

In [58]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [59]:
from IO.save_data_to_db3 import *

In [60]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/3 [00:00<?, ?it/s]

Deleting all datasets for PMID 30377274...
Inserting the new data...


100%|██████████| 3/3 [00:15<00:00,  5.22s/it]

Updating the data_modified_on field...
